# Homework 6: Neural Networks and Deep Learning

In this homework, we'll build a model for classifying various hair types. For this, we will use the Hair Type dataset that was obtained from [Kaggle](https://www.kaggle.com/datasets/kavyasreeb/hair-type-dataset) and slightly rebuilt.

You can download the target dataset for this homework from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip):

```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
unzip data.zip
```

The dataset is split into train and test dataset. Use train dataset to train the model and test dataset for validation.

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch.

### Data Preparation
The dataset contains around 1000 images of hairs in the separate folders for training and test sets.


### Reproducibility

Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs.

Therefore, in this homework we suggest to set the random number seed generators by:

In [47]:
import torch.nn as nn
from pathlib import Path

In [48]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Model
For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.

You need to develop the model with following structure:

* The shape for input should be `(3, 200, 200)` (channels first format in PyTorch)
* Next, create a convolutional layer (`nn.Conv2d`):
  * Use 32 filters (output channels)
  * Kernel size should be `(3, 3)` (that's the size of the filter), padding = 0, stride = 1
  * Use `'relu'` as activation
* Reduce the size of the feature map with max pooling (nn.MaxPool2d)
  * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using `flatten` or `view`
* Next, add a `nn.Linear` layer with 64 neurons and `'relu'` activation
Finally, create the `nn.Linear` layer with 1 neuron - this will be the output
  * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use `torch.optim.SGD` with the following parameters:

```python
torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
```

In [49]:
class HairClassifier(nn.Module):
    def __init__(self):
        super(HairClassifier, self).__init__()
        self.conv = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
     

In [50]:
model = HairClassifier()
device = torch.device("cpu")
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

## Question 1
Which loss function you will use?

* nn.MSELoss()
* nn.BCEWithLogitsLoss()
* nn.CrossEntropyLoss()
* nn.CosineEmbeddingLoss()

In [51]:
criterion = nn.BCEWithLogitsLoss()

## Question 2
What's the total number of parameters of the model? You can use `torchsummary` or count manually.

In PyTorch, you can find the total number of parameters using:

```python
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
```

* 896
* 11214912
* 15896912
* 20073473

In [55]:
sum(p.numel() for p in model.parameters())

20073473

### Generators and Training

For the next two questions, use the following transformation for both train and test sets:


In [56]:
from torchvision import transforms, datasets

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])



* We don't need to do any additional pre-processing for the images.
* Use `batch_size=20`
* Use `shuffle=True` for both training, but False for test.


In [57]:
from torch.utils.data import DataLoader

train_dir = Path("data") / "train"
test_dir = Path("data") / "test"

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_dataset = datasets.ImageFolder(test_dir, transform=train_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(train_dataset, batch_size=20, shuffle=False)


Now fit the model.

You can use this code:


In [58]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6462, Acc: 0.6362, Val Loss: 2.2301, Val Acc: 0.7063
Epoch 2/10, Loss: 0.5475, Acc: 0.7100, Val Loss: 2.2096, Val Acc: 0.7400
Epoch 3/10, Loss: 0.5533, Acc: 0.7250, Val Loss: 1.8838, Val Acc: 0.8013
Epoch 4/10, Loss: 0.4802, Acc: 0.7712, Val Loss: 1.7984, Val Acc: 0.7850
Epoch 5/10, Loss: 0.4334, Acc: 0.8025, Val Loss: 1.2869, Val Acc: 0.8638
Epoch 6/10, Loss: 0.3740, Acc: 0.8325, Val Loss: 1.0510, Val Acc: 0.8988
Epoch 7/10, Loss: 0.2721, Acc: 0.8838, Val Loss: 1.2376, Val Acc: 0.8450
Epoch 8/10, Loss: 0.2478, Acc: 0.9000, Val Loss: 0.8407, Val Acc: 0.9187
Epoch 9/10, Loss: 0.2075, Acc: 0.9200, Val Loss: 0.9176, Val Acc: 0.9062
Epoch 10/10, Loss: 0.1494, Acc: 0.9450, Val Loss: 0.2798, Val Acc: 0.9950


### Question 3
What is the median of training accuracy for all the epochs for this model?

* 0.05
* 0.12
* 0.40
* 0.84

In [60]:
round(np.median(history['acc']), 2)

np.float64(0.82)

### Question 4
What is the standard deviation of training loss for all the epochs for this model?

* 0.007
* 0.078
* 0.171
* 1.710

In [62]:
round(np.std(history['loss']), 3)

np.float64(0.159)

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

```python
transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip()
```

In [64]:
train_transforms_augmented = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

train_dataset_augmented = datasets.ImageFolder(train_dir, transform=train_transforms_augmented)
train_loader_augmented = DataLoader(train_dataset_augmented, batch_size=20, shuffle=True, num_workers=2)

Let's train our model for 10 more epochs using the same code as previously.

Note: make sure you don't re-create the model. we want to continue training the model we already started training.


In [65]:
num_epochs = 10
history_augmented = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader_augmented:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_augmented)
    epoch_acc = correct_train / total_train
    history_augmented['loss'].append(epoch_loss)
    history_augmented['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history_augmented['val_loss'].append(val_epoch_loss)
    history_augmented['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.7218, Acc: 0.6188, Val Loss: 1.5799, Val Acc: 0.8113
Epoch 2/10, Loss: 0.6034, Acc: 0.6850, Val Loss: 1.5373, Val Acc: 0.8125
Epoch 3/10, Loss: 0.5398, Acc: 0.7238, Val Loss: 1.3230, Val Acc: 0.8825
Epoch 4/10, Loss: 0.5217, Acc: 0.7350, Val Loss: 1.8818, Val Acc: 0.7588
Epoch 5/10, Loss: 0.5081, Acc: 0.7675, Val Loss: 1.5289, Val Acc: 0.8087
Epoch 6/10, Loss: 0.4855, Acc: 0.7562, Val Loss: 1.8025, Val Acc: 0.7725
Epoch 7/10, Loss: 0.4678, Acc: 0.7688, Val Loss: 2.1045, Val Acc: 0.7238
Epoch 8/10, Loss: 0.4909, Acc: 0.7700, Val Loss: 1.4076, Val Acc: 0.8612
Epoch 9/10, Loss: 0.4740, Acc: 0.7712, Val Loss: 1.7673, Val Acc: 0.7825
Epoch 10/10, Loss: 0.4557, Acc: 0.7800, Val Loss: 1.8966, Val Acc: 0.7550




### Question 5

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.008
* 0.08
* 0.88
* 8.88

In [66]:
round(np.mean(history_augmented['val_loss']), 2)

np.float64(1.68)

## Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

* 0.08
* 0.28
* 0.68
* 0.98

In [67]:
round(np.average(history_augmented['val_acc'][5:]), 2)

np.float64(0.78)